# Airlines Data Cleaning and Ingestion

The purpose of this notebook is show some of my findings regarding the data, develop data cleaning strategies, and get to look at some of the analytical questions at the bottom.


## Data Loading and Exploration

Let's do some basic exploration.

### Data Loading in production

In the below I'm using Pandas' `read_excel()` for loading directly from Excel sheets. 

For a production-grade setup there should be up-front validation and additional checks to ensure everything has been loaded properly. 

In [1]:
import pandas as pd
import numpy as np

PROJ_PATH = "/data/"

DATA_IN_PREFIX = "orig/"
DATA_OUT_PREFIX = "processed/"

# Load data from Excel sheets.
airlines_raw = pd.read_excel(PROJ_PATH + DATA_IN_PREFIX + "20180102 Airlines.xlsx")
airports_raw = pd.read_excel(PROJ_PATH + DATA_IN_PREFIX + "20180102 Airports.xlsx")
routes_raw = pd.read_excel(PROJ_PATH + DATA_IN_PREFIX + "20180102 Routes.xlsx")
airlines_raw.head()

,Airline ID,Airline Name,Airline Alias,IATA Code,ICAO Code,Airline Callsign,Airline Country,Airline Operational?
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,UK,N


### Types of data issues

We need to distinguish two classes of data problems: Missing and invalid data.

* Looks like the first row of the airlines data is invalid.
* There are missing entries, denoted as **NaN**.
* Other entries have the wrong format, e.g. "\N", or IATA code not according to standard.

Next, let's look at the datatypes and percentages of missing data in all three sets:

In [2]:
## Look at airline types.
print("Data types:\n")
print(airlines_raw.dtypes)

print("\n\n")

# What percentages of actual missing data do we have?
# Note: Missing means NaN; there are other types of flaws in the data which we'll get to later.
def print_perc_missing(df):
    """
    Prints what percentage of data is missing for each column.
    
    CAVEAT: Only checks for nan, not other data defects, e.g. "\\N" entries etc.
    """
    print(round(df.isnull().sum() / df.shape[0] * 100))
    
print("\nAirlines data\n")
print_perc_missing(airlines_raw)

print("\nAirports data\n\n")
print_perc_missing(airports_raw)

print("\nRoutes data\n\n")
print_perc_missing(routes_raw)

Data types:

Airline ID               int64
Airline Name            object
Airline Alias           object
IATA Code               object
ICAO Code               object
Airline Callsign        object
Airline Country         object
Airline Operational?    object
dtype: object




Airlines data

Airline ID               0.0
Airline Name             0.0
Airline Alias            8.0
IATA Code               75.0
ICAO Code                1.0
Airline Callsign        13.0
Airline Country          0.0
Airline Operational?     0.0
dtype: float64

Airports data


Airport ID           0.0
Airport Name         0.0
Airport City         1.0
Airport Country      0.0
IATA Code            0.0
ICAO Code            0.0
Latitude             0.0
Longitude            0.0
Altitude             0.0
Timezone Offset      4.0
DST                  0.0
Timezone             0.0
Unnamed: 12        100.0
Unnamed: 13        100.0
dtype: float64

Routes data


Airline                    0.0
Airline ID                 0.0


### Results

All three data sets have problems.

#### Airlines data
* Airline aliases flawed.
* Most IATA codes are rubbish. Many NaN, and other issues.
* ICAO codes have NaNs and other problems.
* Further, 13% of airline callsigns are flawed.

#### Airports data
* Airport cities seems to be an issue. Not great as we'll need it later on.
* Timezone offset too.
* Some IATA codes are \N --> not NaN! Need to fix this too.
* Definitely need to fix cities if we want to use it as a key.
* There are two unnamed columns, empty columns in the Excel sheet. We will just delete those as they only contain NaNs.

#### Routes data

* The `Codeshare?` column is pretty much useless.
* The `Equipment` column has deeper issues, we'll get to that later.
* The rest doesn't contain missing data, but there are invalid entries.


Lastly, we would like to check the dimensionality of the data:

In [3]:
all_dta = [airlines_raw, airports_raw, routes_raw]

## Print dimensions.
print("\nDimensions...")
list(map(lambda x: print(x.shape), all_dta))


Dimensions...
(6162, 8)
(7184, 14)
(67663, 9)


[None, None, None]

### Results

* Routes has the largest amount of entries, ~68k rows.
* Probably also the most scalable part of the data as new routes will be created, or existing ones updated.
* Need to understand unit of scale in new routes.

## Data Cleaning

For the cleaning process, we'll have use a mechanism that applies different cleaning strategies per each column and data set.

Here are some thoughts:

* Some data, like airlines IATA and ICAO code have a well-defined structure. We can use **regular expressions** to check for problems.
* The same approach could be used to ensure data validity, e.g. for numerical columns.
* For some missing entries, we could **back-fill** data using an API (not implemented).
* In other cases we may have to manually fix the data, so we need to annotate whether there is a problem or not.
* Here is an API that may help with ICAO / IATA issues: https://www.icao.int/safety/iStars/Pages/API-Data-Service.aspx
* In practice, for anything that could be used as an index we need to ensure **uniqueness**.

### ETL in production

The below functions are just examples of how to clean the data in principle. It's neither complete, nor the way this should be done in production.

For production I have created some sketches in the `src/pipeline/` directory using the **Luigi** framework for realizing a batch-based ETL process.

### Annotation vs. automatic filling of missing entries

As said above, for some data we might be able to automatically fill missing entries using APIs or other data sources (e.g. web scraping).

Further, I have chose to introduce marker columns indicating whether there are missing / invalid data present. This can later be used to filter out invalid results in analytical queries.


In [4]:
# Helper functions.
import re

def get_invalid_data(data, column_name, pattern):
    """
    This helper function returns a marker Series where True indicates
    invalid data, if the given regex pattern is NOT matched for a row in <column_name>.
    """
    compiled_pattern = re.compile(pattern)
    invalid_matcher = lambda x: bool(compiled_pattern.match(str(x)))
    return(data[column_name].apply(invalid_matcher))

def create_invalid_col_annotation(data, column_name, invalid_data):
    """
    This helper function creates a new column based on a marker series.
    """
    
    data["invalid_" + column_name] = invalid_data
    return(data)


def generic_str_strategy(data, col):
    """
    Generic cleaning strategy for null / nan data (String columns).
    """
    data.loc[data["invalid_" + col], col] = "MISSING"

### Clean airlines data

In [5]:
def clean_airlines_data(data):
    """
    Invoke cleaning strategy for each column that needs cleaning.
    """
    clean_airlines_iata(data)
    clean_airlines_icao(data)
    clean_airlines_callsign(data)
    clean_airlines_country(data)
    clean_airlines_alias(data)
    
    # Drop first row.
    data = data.drop(0).reset_index(drop=True)
    
    return(data)
     
def clean_airlines_iata(data):
    """
    TBD
    """
    col = "IATA Code"
    create_invalid_col_annotation(data, col, get_invalid_data(data, col, "^(?![A-Z0-9]{2}).*"))
    generic_str_strategy(data, col)

def clean_airlines_icao(data):
    """
    TBD
    """
    col = "ICAO Code"
    create_invalid_col_annotation(data, col, get_invalid_data(data, col, "^(?![A-Z]{3}).*"))
    generic_str_strategy(data, col)


def clean_airlines_callsign(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Airline Callsign"
    invalid_callsigns = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid_callsigns)
    generic_str_strategy(data, col)

def clean_airlines_country(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Airline Country"
    invalid_countries = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid_countries)
    generic_str_strategy(data, col)

def clean_airlines_alias(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Airline Alias"
    invalid_alias = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid_alias)
    generic_str_strategy(data, col)

# NOTE: Here we should also check if the things we use as index, e.g. Airline ID are unique!
airlines = clean_airlines_data(airlines_raw)

### Verify airlines data

In [6]:
# NOTE: Here we should also check if the things we use as index, e.g. Airline ID are unique!
airlines = clean_airlines_data(airlines_raw)

# Now all should be good (in terms of nan / null)
airlines.isnull().sum()

Airline ID                  0
Airline Name                0
Airline Alias               0
IATA Code                   0
ICAO Code                   0
Airline Callsign            0
Airline Country             0
Airline Operational?        0
invalid_IATA Code           0
invalid_ICAO Code           0
invalid_Airline Callsign    0
invalid_Airline Country     0
invalid_Airline Alias       0
dtype: int64

## Cleaning of airports data

We employ the same strategies as above.

**Note**: The most contentious column in this data set is the "City" column, as we'll need it to run analytical queries. Also see comments in function.

In [7]:
def clean_airports_data(data):
    """
    Clean airports data columns.
    """
    clean_airports_iata(data)
    clean_airports_icao(data)
    clean_latitude(data)
    clean_city(data)
    clean_tz_offset(data)
    clean_tz(data)
    clean_dst(data)
    
    # Drop nan columns
    data.drop(data.columns[[12, 13]], axis = 1, inplace = True)
    return(data)


def clean_airports_iata(data):
    """
    TBD
    """
    col = "IATA Code"
    
    create_invalid_col_annotation(data, col, get_invalid_data(data, col, "^(?![A-Z]{3}).*"))
    generic_str_strategy(data, col)    

def clean_airports_icao(data):
    """
    TBD
    """
    col = "ICAO Code"
    create_invalid_col_annotation(data, col, get_invalid_data(data, col, "^(?![A-Z]{4}).*"))
    generic_str_strategy(data, col)    

def clean_city(data):
    """
    This is actually crucial because we want to use City as a key!
    
    Also, this particular column seems to be a mess.
    
    Real-world considerations:
        - UTF conversion issues?
        - Invalid values....
    """
    col = "Airport City"
    invalid_city = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid_city)
    generic_str_strategy(data, col)


def clean_tz_offset(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Timezone Offset"
    invalid_tz_off = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid_tz_off)
    generic_str_strategy(data, col)

def clean_tz(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Timezone"
    invalid_tz = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid_tz)
    generic_str_strategy(data, col)

def clean_dst(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "DST"
    invalid_dst = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid_dst)
    generic_str_strategy(data, col)

def clean_latitude(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Latitude"
    inv = data[col].isnull()
    create_invalid_col_annotation(data, col, inv)
    generic_str_strategy(data, col)


### Verify airports data

In [8]:
airports = clean_airports_data(airports_raw)
airports.isnull().sum()

Airport ID                 0
Airport Name               0
Airport City               0
Airport Country            0
IATA Code                  0
ICAO Code                  0
Latitude                   0
Longitude                  0
Altitude                   0
Timezone Offset            0
DST                        0
Timezone                   0
invalid_IATA Code          0
invalid_ICAO Code          0
invalid_Latitude           0
invalid_Airport City       0
invalid_Timezone Offset    0
invalid_Timezone           0
invalid_DST                0
dtype: int64

## Cleaning routes data

One of the issues here is the **equipment column** which may hold multiple entries. 

As we want to be able to query for equipment type, the strategy is to create one boolean-type column for each equipment type indicating whether this aircraft is being used on the present route, or not (True / False).

There are other issues similar to the other data sets (like `\N`) etc.

In [9]:
def clean_routes_data(data):
    """
    Invokes all the column cleaning strategies for the routes data.
    """
    data = clean_equipment(data)
    clean_airline(data)
    clean_destination_airport(data)
    clean_codeshare(data)
    clean_no_stops(data)
    data = clean_source_airport_id(data)
    data = clean_destination_airport_id(data)
    return(data)
    

def clean_airline(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Airline"
    invalid = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid)
    generic_str_strategy(data, col)    
    
def clean_destination_airport(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Destination Airport"
    invalid = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid)
    generic_str_strategy(data, col) 
    
def clean_codeshare(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "Codeshare?"
    invalid = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid)
    generic_str_strategy(data, col)  

def clean_no_stops(data):
    """
    FIXME: This only filters out nan values, not *all* incorrect values!
    """
    col = "No Stops"
    invalid = data[col].isnull()
    create_invalid_col_annotation(data, col, invalid)
    generic_str_strategy(data, col)  

def clean_equipment(data):
    """
    This function cleans the equipment column.
    """
    
    # Convert this column to a string type.
    data = data.astype({"Equipment": np.str})
    
    # Flatten a nested list.
    # FIXME: Should consider using itertools for Python >= 2.6
    # https://docs.python.org/3/library/itertools.html#itertools.product
    flatten = lambda l: [item for sublist in l for item in sublist]

    equip_raw = data["Equipment"]

    # Generate a list of unique equpment types.
    # FIXME: Might need to consider more than one space; remove empty strings / nan.
    equip_raw = [e.split(" ") for e in equip_raw]    
    equip = np.unique(flatten(equip_raw))
    equip[(equip == "") |  (equip == "nan")] = "MISSING"
    
    # Generate column names.
    prepend_eq = np.vectorize(lambda x: "eq_" + x)
    equip_cols = prepend_eq(equip)
    
    # Now add a boolean column for each eq. type.
    tmp = data.reindex(columns = equip_cols)
    tmp.fillna(value = False, inplace = True)
    tmp = tmp.astype(np.bool)
    
    # Join
    data.join(tmp)
    
    # Set to true if equip is present.
    # FIXME: Perhaps not the most elegant solution.
    for col in equip:
        data["eq_" + col] = data["Equipment"].str.contains(col)
    
    return(data)
    
def clean_airport_id(data, col_name):
    """
    Generic function to clean airport IDs.
    """
    non_num = re.compile("^(?![0-9]+).*")
    invalid_ids = lambda x: bool(non_num.match(str(x)))
    inv_ids = data[col_name].apply(invalid_ids)
    data.loc[inv_ids, col_name] =  "-1"
    # Convert to int!    

def clean_source_airport_id(data):
    """
    Clean the Source ID column, so we can use it as a key.
    """
    clean_airport_id(data, "Source Airport ID")
    return(data.astype({"Source Airport ID": np.int}))
    
def clean_destination_airport_id(data):
    """
    Clean the Destination ID column, so we can use it as a key.
    """
    clean_airport_id(data, "Destination Airport ID")
    return(data.astype({"Destination Airport ID": np.int}))

### Verify routes data

In [10]:
routes = clean_routes_data(routes_raw)

routes.isnull().sum()

Airline                        0
Airline ID                     0
Source Airport                 0
Source Airport ID              0
Destination Airport            0
Destination Airport ID         0
Codeshare?                     0
No Stops                       0
Equipment                      0
eq_MISSI                       0
eq_100                         0
eq_141                         0
eq_142                         0
eq_143                         0
eq_146                         0
eq_310                         0
eq_313                         0
eq_318                         0
eq_319                         0
eq_320                         0
eq_321                         0
eq_326                         0
eq_329                         0
eq_32A                         0
eq_32B                         0
eq_32C                         0
eq_32S                         0
eq_330                         0
eq_332                         0
eq_333                         0
          

## Data saving / ingestion

We can now export the cleaned data either as CSV files or quickly load them into a Postgres DB.


### Data Warehouse in production

Again, the chosen set-up here is clearly not how to deploy this in production. Check the project's `README.md` and the notes in `src/db` for some technology suggestions.

In [11]:
!conda install psycopg2 -y

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    krb5:     1.14.2-hcdc1b81_6     
    libpq:    9.6.6-h4e02ad2_0      
    psycopg2: 2.7.3.2-py36h2b1659c_0

The following packages will be UPDATED:

    anaconda: 5.0.1-py36hd30a520_1   --> custom-py36hbbc8b67_0
    conda:    4.3.30-py36h5d9f9f4_0  --> 4.4.8-py36_0         
    openssl:  1.0.2l-h077ae2c_5      --> 1.0.2n-hb7f436b_0    
    pycosat:  0.6.2-py36h1a0ea17_1   --> 0.6.3-py36h0a5515d_0 

openssl-1.0.2n 100% |################################| Time: 0:00:08 405.40 kB/s
krb5-1.14.2-hc 100% |################################| Time: 0:00:02 531.27 kB/s
libpq-9.6.6-h4 100% |################################| Time: 0:00:00   1.77 MB/s
anaconda-custo 100% |################################| Time: 0:00:00  13.75 MB/s
psycopg2-2.7.3 100% |################################| Time: 0:00:00   1.77 MB/s
pycosat-0.6

In [12]:
# Save processed data to CSV.

# Un-comment to enable.
#
# airlines.to_csv(PROJ_PATH + DATA_OUT_PREFIX + "airlines.csv", index = False)
# airports.to_csv(PROJ_PATH + DATA_OUT_PREFIX + "airports.csv", index = False)
# routes.to_csv(PROJ_PATH + DATA_OUT_PREFIX + "routes.csv", index = False)


#
# Ingest directly to database.
#
# FIXME: Should consider converting column names to exclude whitespaces.
# FIXME: Supply data schema.

# Note: May need to install additional packages:
# !conda install psycopg2
# Note: Ensure DB has been created beforehand.

from sqlalchemy import create_engine

PSQL_HOST = "db"
PSQL_USER = "postgres"
PSQL_PW = "admin"
PSQL_PORT = "5432"
DB_NAME = "airlines_data"

DB_URL = "postgresql://" + PSQL_USER + ":" + PSQL_PW + "@" + PSQL_HOST + ":" + PSQL_PORT + "/" + DB_NAME

print("Connecting to PSQL URL: " + DB_URL)

# Create connection.
engine = create_engine(DB_URL)

# Dump data using default schema and name.
airlines.to_sql(name = "airlines", con = engine, if_exists = "replace")
airports.to_sql(name = "airports", con = engine, if_exists = "replace")

# This one will take a bit of time.
# FIXME: Supply schema!
routes.to_sql(name = "routes", con = engine, if_exists = "replace")

# Close connection.
con = engine.raw_connection()
con.close()

Connecting to PSQL URL: postgresql://postgres:admin@db:5432/airlines_data


## Data Analysis

In the below I get to some of the questions regarding the data.

The results are obtained using Pandas Dataframe filtering. The same could be achieved using SQL queries, see the `src/db` folder for examples like:


```sql
-- How many routes do we have out of London, UK?
select count(*) from airports a 
	join routes r on a."Airport ID" = r."Source Airport ID" 
	where a."Airport ID" in
	(select "Airport ID" from airports where "Airport City" like '%London' and "Airport Country" like '%United Kingdom')
```

### 1. Which aircraft type is the most common on routes out of London?

I'm assuming London, UK (there is a London in the UK, Canada and US)




In [13]:
## Firstly, has any of the missing cities to do with London?
airports["Airport Name"].str.contains("London")
london_in_name = airports["Airport Name"].str.contains("London")
#print(airports[london_in_name])
## Seemingly so, but wouldn't work with any city.
## Bottom line is City really needs to be cleaned properly.


## Get all the routes out of London.

## Issue here: Both City and Country might be flawed!
london_airport_ids = airports[(airports["Airport City"] == "London") & (airports["Airport Country"] == "United Kingdom")]["Airport ID"]

## Get all the routes out of London.
routes_out_of_london = routes[routes["Source Airport ID"].isin(list(london_airport_ids))]

## Get the most frequently used equipment type.
most_freq = routes_out_of_london.filter(regex = "^eq_.*").sum().sort_values(ascending = False)

## RES: A320
print(most_freq.head())

eq_320    352
eq_319    303
eq_738    162
eq_777    134
eq_321    128
dtype: int64


### Result

Looks like the Airbus A320 is the most frequent equipment.

### 2. Which airline has the most routes into JFK?


In [14]:
## Need ID of destination JFK. We will use IATA.
## NOTE: This means IATA must be ok.
jfk_id = int(airports[(airports["IATA Code"] == "JFK")]["Airport ID"].values)

## Get all the airlines IDs with jfk_id as destination.
airline_ids_into_jfk = routes[routes["Destination Airport ID"] == jfk_id]["Airline ID"]

## Get ID with most flights.
most_routes_id = airline_ids_into_jfk.value_counts().index[0]

## Look up name.
## RES: Delta Airlines.
print(airlines[airlines["Airline ID"] == most_routes_id]["Airline Name"])

2007    Delta Air Lines
Name: Airline Name, dtype: object


### Result

Delta Airlines